LOADING AND VISUALIZATION OF RAW DATA

In [1]:
# Importing necessary libraries for analysis and visualization
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import seaborn as sns
import re

# Set plotting font to Segoe UI (font used for the report)
plt.rcParams['font.family'] = 'Segoe UI'

# Load the dataset
df = pd.read_csv("aircraft_data.csv")

#print(df.shape)       # Shows DataFrame size
#print(df.info())      # Shows the size, columns, and data types for each variable


# # Represent a heat map for null values

# custom_cmap = ListedColormap(['#DAE3F3', '#00205B'])  # Gray for NaNs, Blue for values
# plt.figure(figsize=(10, 6))
# sns.heatmap(df.notnull(), cbar=False, cmap=custom_cmap)
# plt.title("Null Values Heat Map")
# plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees, align to the right
# y_labels = range(0, len(df), max(1, len(df) // 12))  # Display ~10 evenly spaced labels
# plt.yticks(y_labels, [df.index[i] for i in y_labels], rotation=0)  # Keep y-axis labels horizontal
# plt.show()

c:\Users\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


GENERAL DATA CLEANING

In [2]:
# DELETION OF VALUES THAT ARE NOT PROVIDING INFORMATION (EG. Not Liste, TBA, etc)
df.replace({'Not Listed': np.nan, 'TBA': np.nan, 'tba': np.nan, 'tbc': np.nan, 'None': np.nan,'TBD': np.nan, '-': np.nan, 'n/a': np.nan, 'NA': np.nan, 'TBD  ':np.nan, 'Not Specified':np.nan}, inplace=True)


# DELETION OF REPEATED ROWS

# Completely repeated
print(f'Dataframe size before the deletion of repeated rows: {df.shape}')
df.drop_duplicates(inplace=True)
print(f'Dataframe size after the deletion of repeated rows: {df.shape}')
#Conclusion: There were a total of 3 rows which were initially completely repeated 



# 'S/N' having repeated values, in this case only the row with less NaN is kept, being completed with information from other rows in case of missing values
print(f"The size of the Dataset before the deletion of rows with repeated 'S/N': {df.shape}")
# 1: Separate NaN and values in'S/N' column
nan_rows_sn = df[df['S/N'].isna()]
non_nan_rows_sn = df[df['S/N'].notna()]

# Identify and display repeated 'S/N' values
repeated_sn = non_nan_rows_sn['S/N'].value_counts()
repeated_sn = repeated_sn[repeated_sn > 1].index.tolist()
print("Valores de 'S/N' que se encontraron repetidos inicialmente:", repeated_sn)

# 2: Function definition
def fill_na_with_others(group):
    # Group rows with same 'S/N' and order them starting with the row with less NaN
    group = group.loc[group.apply(lambda row: row.isna().sum(), axis=1).sort_values().index]
    
    # Take the first row (with less NaNs) as base
    base_row = group.iloc[0].copy()
    
    # Fill NaNs in base row using the rest of the rows from the group following its order
    for _, row in group.iterrows():
        for col in base_row.index:
            if pd.isna(base_row[col]) and not pd.isna(row[col]):
                base_row[col] = row[col]
    
    return base_row

# 3: Apply the function for each group of 'S/N' and create a DataFrame with the completed rows
filled_rows = non_nan_rows_sn.groupby('S/N').apply(fill_na_with_others).reset_index(drop=True)

# 4: Join rows with NaNs in 'S/N' and the completed rows
df = pd.concat([filled_rows, nan_rows_sn], ignore_index=True)

# Check the size of the Dataset after the deletion of rows
print(f"The size of the Dataset after the deletion of rows with repeated 'S/N': {df.shape}")
#Conclusion: There were a total of 147 rows which initially had repeated S/N value



# Repeate the same process for REG column
print(f"The size of the Dataset before the deletion of rows with repeated 'REG': {df.shape}")
# 1: Separate NaN and values in'REG' column
nan_rows_sn = df[df['REG'].isna()]
non_nan_rows_sn = df[df['REG'].notna()]

# Identify and display repeated 'REG' values
repeated_sn = non_nan_rows_sn['REG'].value_counts()
repeated_sn = repeated_sn[repeated_sn > 1].index.tolist()
print("Valores de 'REG' que se encontraron repetidos inicialmente:", repeated_sn)

# 2: Apply previous defined function for each group of 'REG' and create a DataFrame with the completed rows
filled_rows = non_nan_rows_sn.groupby('REG').apply(fill_na_with_others).reset_index(drop=True)

# 3: Join rows with NaNs in 'REG' and the completed rows
df = pd.concat([filled_rows, nan_rows_sn], ignore_index=True)

# Check the size of the Dataset after the deletion of rows
print(f"The size of the Dataset after the deletion of rows with repeated 'REG': {df.shape}")
#Conclusion: There were a total of 17 rows which initially had repeated REG value



# DELETION OF NON-RELEVANT COLUMNS 

# No containing relevant information for the problem to solve: Too soon to konw
# #db.drop(columns = '[insert colum]', axis=1, inplace=True)

# Having an unique value for all of the rows: Not applicable for the datset under study 

#Having different values for each of the rows: 'S/N' and 'REG'
unique_columns = [col for col in df.columns if df[col].dropna().nunique() == len(df[col].dropna())]
print("Columnas con valores únicos (sin contar NaN):", unique_columns)
df.drop(columns = unique_columns, axis=1, inplace=True)


Dataframe size before the deletion of repeated rows: (2530, 18)
Dataframe size after the deletion of repeated rows: (2527, 18)
The size of the Dataset before the deletion of rows with repeated 'S/N': (2527, 18)
Valores de 'S/N' que se encontraron repetidos inicialmente: ['36', '1', '210', '42', '62.112', '56', '18253712', '18', '3601', 'L#1558I', '4879', '2013', '24', '2045', '42.AC011', '1481', '1821', '4692179', '3835', '1612', '196', '13276', '4697241', '3257434', '32-0009', '124', '96', '126', '1509', '1500', '1617', '3428', '3236', '42058', 'T20608487', '310R0044', '41568', '20800413', '720', '24-0944', '1976', '3667', '3858', '2017', 'T20609088', '20', '20800402', '1695', '1376', '4745', '62.C021', '75-8233', 'P3351', '1478', '1874', '444/2019', '1917', '1150-2014', '4294', '54641', '2009', '2953', '3287', '705', '892', '924', '700', '4076', '4692023', '4636401', '349', '6', '670', '581', '599', '676', '3167', '41', '50000064', '2059', '1144', '341', '1219', '4697601', '20800309'

C:\Users\Ángela Martín Milán\AppData\Local\Temp\ipykernel_40288\2833007324.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_rows = non_nan_rows_sn.groupby('S/N').apply(fill_na_with_others).reset_index(drop=True)


The size of the Dataset after the deletion of rows with repeated 'S/N': (2380, 18)
The size of the Dataset before the deletion of rows with repeated 'REG': (2380, 18)
Valores de 'REG' que se encontraron repetidos inicialmente: ['N555HJ', 'OK-WWD04', 'N25646', 'N557BL', 'N571LL', 'G-HEWZ', 'N121SP', 'N74WY', 'G-BXAY', 'N524MR', 'N262MC', 'N125MM', 'N525DE', 'N32V', 'N20GT', 'N686AG', 'N9193Q']
The size of the Dataset after the deletion of rows with repeated 'REG': (2363, 18)
Columnas con valores únicos (sin contar NaN): ['S/N', 'REG']


C:\Users\Ángela Martín Milán\AppData\Local\Temp\ipykernel_40288\2833007324.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_rows = non_nan_rows_sn.groupby('REG').apply(fill_na_with_others).reset_index(drop=True)


ELEMENTARY FEATURE TRANSFORMATIONS

In [3]:
# CONDITION

# General view
# print('Values that the Condition variable takes:')
# print(df['Condition'].value_counts(dropna=False))
# Main remarks: This variable includes 3 values and a high values of NaN.

# Ensure this variable is detected as categoric
df['Condition'] = df['Condition'].astype('category')

In [4]:
# PRICE

# General view
# print('Values that the Price variable takes:')
# print(df['Price'].value_counts(dropna=False))
# Main remarks: This variable includes also the currency before the figure, only numbers will be kept and letters will be extracted into other column
# as can be useful for 'Currency'. Commas indicating thousands will be removed. NaN values are present.


# Function to keep only numbers
def extract_numbers(precio):
    if pd.isna(precio):  # Check if the value is NaN
        return np.nan
    match = re.search(r'[\d,]+(?:\.\d+)?', precio)
    if match:
        return float(match.group(0).replace(',', ''))
    return np.nan  # NaN if no numbers are found

# Function to keep only letters previous to the first figure
def extract_text(precio):
    if pd.isna(precio):  # Check if the value is NaN
        return np.nan
    if not isinstance(precio, str):  # Ensure that 'precio' is a string
         precio = str(precio)  # Convert to string if it's not
    match = re.search(r'^(.*?)(\d)', precio)
    if match:
        return match.group(1).strip()
    return None  # None if no text is found

# Apply functions
df['text'] = df['Price'].apply(extract_text)  # Saved into a new column named 'text'
#print(df['text'].value_counts(dropna=False))
df['Price'] = df['Price'].apply(extract_numbers)



# Show resultant DataFrame
#print(db['Price'].value_counts(dropna=False))


# Ensure this variable is detected as Float32
df['Price'] = df['Price'].astype('float32')


In [5]:
# CURRENCY

# General view
#print('Values that the Currency variable takes:')
# print(df['Currency'].value_counts(dropna=False))
# Main remarks: This variable includes a high number of NaN values, NaN will be completed from the currency specified into the Price column

# Ensure this variable is detected as categoric


In [6]:
# CATEGORY

# General view
# print('Values that the Category variable takes:')
# print(df['Category'].value_counts(dropna=False))
# Main remarks: This variable includes 13 different values with no NaN, some of them are referring to the same category of aircraft but are 
# diferently written (some typos as well). As well they are not representing the same aircraft classification, so additional columns will be 
# created to ensure the same level of information.

# Firstly let's group values that are referring to the same concept
df['Category'] = df['Category'].str.replace('Single Piston','Single Engine Piston', regex=False)         # Single Piston means the same as Single Engine Piston
df['Category'] = df['Category'].str.replace('Twin Piston','Multi Engine Piston', regex=False)            # Twin Piston means the same as Multi Engine Piston
df['Category'] = df['Category'].str.replace('Turboprops','Turboprop', regex=False)                       # Turboprops means the same as Turboprop
df['Category'] = df['Category'].str.replace('Gliders | Sailplanes','Gliders/Sailplanes', regex=False)    # Change  '|' separation mark by '/'


# Fix detected errors
# The Boeing PT-17 model is labeled as Private Jet, but it's actually a vintage military aircraft
df.loc[df['Model'] == 'PT-17', ['Category']] = 'Military/Classic/Vintage'


# From this first Category column, three more columns will be added representing the same level of classification: Category, Propulsion and No. Engines

# Aircraft Category
df.loc[df['Category'].isin(['Single Engine Piston', 'Multi Engine Piston', 'Private Jets', 'Turboprop']), 'category'] = 'Airplane'
df.loc[df['Category'].isin(['Turbine Helicopters', 'Piston Helicopters', 'Gyrocopter']), 'category'] = 'Rotorcraft'
df.loc[df['Category'].isin(['Military/Classic/Vintage']), 'category'] = 'Military/Classic/Vintage'
df.loc[df['Category'].isin(['Ultralight']), 'category'] = 'Ultralight'
df.loc[df['Category'].isin(['Gliders/Sailplanes']), 'category'] = 'Motor Gliders/Sailplanes' # All are motor gliders

#print(df['category'].value_counts(dropna=False))


#Aircraft propulsion:
df.loc[df['Category'].isin(['Single Engine Piston', 'Multi Engine Piston']), 'Propulsion'] = 'Piston Engine'
df.loc[df['Category'].isin(['Private Jets']), 'Propulsion'] = 'Turbofan'
df.loc[df['Category'].isin(['Turboprop']), 'Propulsion'] = 'Turboprop'
df.loc[df['Category'].isin(['Turbine Helicopters']), 'Propulsion'] = 'Turboshaft' 
df.loc[df['Category'].isin(['Piston Helicopters']), 'Propulsion'] = 'Piston Engine'
df.loc[df['Category'].isin(['Gyrocopter']), 'Propulsion'] = 'Gyrocopter'
df.loc[df['Category'].isin(['Military/Classic/Vintage']), 'Propulsion'] = np.nan
df.loc[df['Category'].isin(['Ultralight']), 'Propulsion'] = np.nan
df.loc[df['Category'].isin(['Gliders/Sailplanes']), 'Propulsion'] = np.nan # Piston engines

#print(df['Propulsion'].value_counts(dropna=False))


# No. Engines - ASK
df.loc[df['Category'].isin(['Single Engine Piston', 'Gyrocopter']), 'No. Engines'] = 1
df.loc[df['Category'].isin(['Multi Engine Piston']), 'No. Engines'] = 2

#print(df['No. Engines'].value_counts(dropna=False))


# Ensure these columns are detected as categoric and integer32
df['category'] = df['category'].astype('category')
df['Propulsion'] = df['Propulsion'].astype('category')
df['No. Engines'] = df['No. Engines'].astype('Int32')

print(df['No. Engines'].value_counts(dropna=False))


No. Engines
1       1312
<NA>     744
2        307
Name: count, dtype: Int64


In [7]:
# YEAR

# General view
# print('Values that the Year variable takes:')
# print(df['Year'].value_counts(dropna=False))
# Main remarks: This variable includes many different values with no NaN, this variable is identified as categoric, it will be converted into 
# numeric (integer)


# Convert from categoric to numeric values (integer): Int32
df['Year'] = df['Year'].astype('Int32')

#print(df.info())

In [8]:
# MAKE

# General view
# print('Values that the Make variable takes:')
# print(df['Make'].value_counts(dropna=False))
# Main remarks: This variable includes many different values with no NaN, the number of different values is too high that they can not be checked
# just format improvements will be made

# Unify value formats by using uppercase for only the first letter of each word
df['Make'] = df['Make'].str.title()
# Change to data type to String
df["Make"] = df["Make"].astype("string")

#print(df['Make'].value_counts(dropna=False))



In [9]:
# MODEL

# General view
# print('Values that the Model variable takes:')
# print(df['Model'].value_counts(dropna=False))
# Main remarks: This variable includes many different values with no NaN, the number of different values is too high that they can not be checked
# just format improvements will be made


# Unify value formats by using uppercase for only the first letter of each word
df['Model'] = df['Model'].str.title()
# Change to data type to String
df["Model"] = df["Model"].astype("string")

# #Para comprobar:
#pd.set_option('display.max_rows', None)
#print(db['Model'].value_counts(dropna=False))

In [10]:
# LOCATION

# General view
# print('Values that the Location variable takes:')
# print(df['Location'].value_counts(dropna=False))
# Main remarks: This variable includes many different values and NaN. In general it can be observed that there exists three different types in which
# Location is reported, and normally the city, state and country are provided. Due to this, four additional columns will be added: 'City',
# 'State', 'Country' and 'Continent'

# Function to extract city, state, country, and continent depending on the format in which Location is expressed
def extract_location(location):
    # Check that the cell is a string
    if isinstance(location, str):
        # Split the cell into lines
        lines = location.split('\n')
        
        # Case 1: Three lines, format city-state-country
        if len(lines) == 3:
            city = lines[0].replace(',', '').strip()
            state = re.sub(r'\s+', '', lines[1]) if lines[1].strip() else np.nan
            country = re.sub(r'\s+', '', lines[2])
            continent = np.nan
            return pd.Series([city, state, country, continent])

        # Case 2: Two lines, format state-country
        elif len(lines) == 2:
            city = np.nan
            state = re.sub(r'\s+', '', lines[0])  # Remove spaces in state
            country = re.sub(r'\s+', '', lines[1])  # Remove spaces in country
            continent = np.nan
            return pd.Series([city, state, country, continent])
        
        # Case 3: One line with commas, format continent-country-state
        elif len(lines) == 1 and ',' in lines[0]:
            # Split the line into parts using commas
            parts = lines[0].split(',')
            
            # Extract continent (text before the first comma)
            continent = parts[0].strip()
            
            # Extract country and state (if exists)
            if len(parts) > 1:
                country_and_state = parts[1].strip()
                
                # Check if there's a hyphen to split country and state
                if '-' in country_and_state:
                    country, state = map(str.strip, country_and_state.split('-', 1))
                else:
                    country = country_and_state
                    state = np.nan
            else:
                country = np.nan
                state = np.nan
            
            # City doesn't apply in this format
            city = np.nan
            return pd.Series([city, state, country, continent])
    
    # Return NaN for each field if format is invalid
    return pd.Series([np.nan, np.nan, np.nan, np.nan])

# Apply the function and create new columns
df[['City', 'State', 'Country', 'Continent']] = df['Location'].apply(extract_location)


# Function to verify if a cell matches any of these formats to get values not presenting the three formats described above
def check_location_format(location):
    if isinstance(location, str):
        lines = location.split('\n')
        
        # Check the first format: three lines
        if len(lines) == 3:
            city = lines[0].replace(',', '').strip()
            state = re.sub(r'\s+', '', lines[1]) if lines[1].strip() else ""
            country = re.sub(r'\s+', '', lines[2])
            if city and country:
                return True  # Matches the three-line format
            

        # Check the second format: two lines, state-country
        elif len(lines) == 2:
            state = re.sub(r'\s+', '', lines[0])
            country = re.sub(r'\s+', '', lines[1])
            if state and country:
                return True  # Matches the two-line format
            

        # Check the third format: one line with commas
        elif len(lines) == 1 and ',' in lines[0]:
            parts = lines[0].split(',')
            continent = parts[0].strip()
            if len(parts) > 1:
                country_and_state = parts[1].strip()
                if '-' in country_and_state or country_and_state:
                    return True  # Matches the one-line with commas format
    
    # Doesn't match any of the formats
    return False

# Filter rows that don't match the formats
invalid_locations = df[~df['Location'].apply(check_location_format)]

# Display results
# print("Cells that don't match any of the formats:")
# print(invalid_locations[['Location']])

# Fill cells that don't match any of the three described formats
df.loc[df['Location'] == 'DEU', ['Country']] = 'Germany'
df.loc[df['Location'] == 'ESP', ['Country']] = 'Spain'
df.loc[df['Location'] == 'FRA', ['Country']] = 'France'
df.loc[df['Location'] == 'CHE', ['Country']] = 'Switzerland'
df.loc[df['Location'] == 'ITA', ['Country']] = 'Italy'
df.loc[df['Location'] == 'AUS', ['Country']] = 'Australia'
df.loc[df['Location'] == 'CAN', ['Country']] = 'Canada'
df.loc[df['Location'] == 'CHL', ['Country']] = 'Chile'
df.loc[df['Location'] == 'USA', ['Country']] = 'USA'
df.loc[df['Location'] == 'Europe', ['Continent']] = 'Europe'
df.loc[df['Location'] == 'Asia', ['Continent']] = 'Asia'
df.loc[df['Location'] == 'Temple', ['City']] = 'Temple'
df.loc[df['Location'] == 'Temple', ['State']] = 'CA'
df.loc[df['Location'] == 'Temple', ['Country']] = 'USA'
df.loc[df['Location'] == 'Bethany', ['City']] = 'Bethany'
df.loc[df['Location'] == 'Bethany', ['State']] = 'OK'
df.loc[df['Location'] == 'Bethany', ['Country']] = 'USA'
df.loc[df['Location'] == 'Nashville', ['City']] = 'Nashville'
df.loc[df['Location'] == 'Nashville', ['State']] = 'TN'
df.loc[df['Location'] == 'Nashville', ['Country']] = 'USA'
df.loc[df['Location'] == 'Madison', ['City']] = 'Madison'
df.loc[df['Location'] == 'Madison', ['State']] = 'WI'
df.loc[df['Location'] == 'Madison', ['Country']] = 'USA'
df.loc[df['Location'] == 'Ft. Lauderdale', ['City']] = 'Ft. Lauderdale'
df.loc[df['Location'] == 'Ft. Lauderdale', ['State']] = 'FL'
df.loc[df['Location'] == 'Ft. Lauderdale', ['Country']] = 'USA'
df.loc[df['Location'] == 'KCHD', ['City']] = 'Chandler'
df.loc[df['Location'] == 'KCHD', ['State']] = 'AZ'
df.loc[df['Location'] == 'KCHD', ['Country']] = 'USA'
df.loc[df['Location'] == 'Newnan', ['City']] = 'Newnan'
df.loc[df['Location'] == 'Newnan', ['State']] = 'GA'
df.loc[df['Location'] == 'KNewnan', ['Country']] = 'USA'
df.loc[df['Location'] == 'Corvallis', ['City']] = 'Corvallis'
df.loc[df['Location'] == 'Corvallis', ['State']] = 'OR'
df.loc[df['Location'] == 'Corvallis', ['Country']] = 'USA'
df.loc[df['Location'] == 'Cameron', ['City']] = 'Cameron'
df.loc[df['Location'] == 'Cameron', ['State']] = 'TX'
df.loc[df['Location'] == 'Cameron', ['Country']] = 'USA'
df.loc[df['Location'] == 'Heber City', ['City']] = 'Heber City'
df.loc[df['Location'] == 'Heber City', ['State']] = 'UT'
df.loc[df['Location'] == 'Heber City', ['Country']] = 'USA'
df.loc[df['Location'] == '73501', ['City']] = 'Lawton'
df.loc[df['Location'] == '73501', ['State']] = 'OK'
df.loc[df['Location'] == '73501', ['Country']] = 'USA'
df.loc[df['Location'] == 'Jacksonville - CRG', ['City']] = 'Jacksonville'
df.loc[df['Location'] == 'Jacksonville - CRG', ['State']] = 'FL'
df.loc[df['Location'] == 'Jacksonville - CRG', ['Country']] = 'USA'
df.loc[df['Location'] == 'Jacksonville', ['City']] = 'Jacksonville'
df.loc[df['Location'] == 'Jacksonville', ['State']] = 'FL'
df.loc[df['Location'] == 'Jacksonville', ['Country']] = 'USA'
df.loc[df['Location'] == 'Alexandria', ['City']] = 'Alexandria'
df.loc[df['Location'] == 'Alexandria', ['State']] = 'MN'
df.loc[df['Location'] == 'Alexandria', ['Country']] = 'USA'
df.loc[df['Location'] == 'Lodi', ['City']] = 'Lodi'
df.loc[df['Location'] == 'Lodi', ['State']] = 'CA'
df.loc[df['Location'] == 'Lodi', ['Country']] = 'USA'
df.loc[df['Location'] == 'Spokane', ['City']] = 'Spokane'
df.loc[df['Location'] == 'Spokane', ['State']] = 'WA'
df.loc[df['Location'] == 'Spokane', ['Country']] = 'USA'
df.loc[df['Location'] == 'North America + Canada', ['Continent']] = 'North America'
df.loc[df['Location'] == 'AUBURN', ['City']] = 'Auburn'
df.loc[df['Location'] == 'AUBURN', ['State']] = 'AL'
df.loc[df['Location'] == 'AUBURN', ['Country']] = 'USA'
df.loc[df['Location'] == 'Kankakee', ['City']] = 'Kankakee'
df.loc[df['Location'] == 'Kankakee', ['State']] = 'IL'
df.loc[df['Location'] == 'Kankakee', ['Country']] = 'USA'
df.loc[df['Location'] == 'Oxford (KHNZ)', ['City']] = 'Oxford'
df.loc[df['Location'] == 'Oxford (KHNZ)', ['State']] = 'NC'
df.loc[df['Location'] == 'Oxford (KHNZ)', ['Country']] = 'USA'
df.loc[df['Location'] == 'Madras', ['City']] = 'Madras'
df.loc[df['Location'] == 'Madras', ['Country']] = 'India'
df.loc[df['Location'] == 'Madras', ['Continent']] = 'Asia'
df.loc[df['Location'] == 'Durango', ['City']] = 'Durango'
df.loc[df['Location'] == 'Durango', ['Country']] = 'Spain'
df.loc[df['Category'].isin(['DEU', 'ESP', 'FRA', 'CHE', 'ITA']), 'Continent'] = 'Europe'
df.loc[df['Category'].isin(['AUS']), 'Continent'] = 'Oceania'
df.loc[df['Category'].isin(['CHL']), 'Continent'] = 'South America'

# Standardize some cell values for consistency
df['Country'] = df['Country'].str.replace('England', 'United Kingdom', regex=False)
df['Continent'] = df['Continent'].str.replace('Australia & NZ', 'Oceania', regex=False)
df['Continent'] = df['Continent'].str.replace('North America + Canada', 'North America', regex=False)
df.loc[df['City'] == 'Canada', ['City']] = np.nan
df.loc[df['Location'] == 'Canada', ['Country']] = 'Canada'
df.loc[df['Country'] == 'USA', ['Country']] = 'United States'
df.loc[df['Country'] == 'MEX', ['Country']] = 'Mexico'
df.loc[df['Country'] == 'AUS', ['Country']] = 'Australia'
df.loc[df['Country'] == 'CAN', ['Country']] = 'Canada'
df.loc[df['Country'] == 'CHE', ['Country']] = 'Switzerland'
df.loc[df['Country'] == 'ZAF', ['Country']] = 'South Africa'
df.loc[df['Country'] == 'BEL', ['Country']] = 'Belgium'
df.loc[df['Country'] == 'POL', ['Country']] = 'Poland'
df.loc[df['Country'] == 'NGA', ['Country']] = 'Nigeria'
df.loc[df['Country'] == 'GBR', ['Country']] = 'United Kingdom'
df.loc[df['Country'] == 'SRB', ['Country']] = 'Serbia'
df.loc[df['Country'] == 'NLD', ['Country']] = 'Netherlands'
df.loc[df['Country'] == 'CZE', ['Country']] = 'Czech Republic'
df.loc[df['Country'] == 'ARG', ['Country']] = 'Argentina'
df.loc[df['Country'] == 'FRA', ['Country']] = 'France'
df.loc[df['Country'] == 'URY', ['Country']] = 'Uruguay'
df.loc[df['Country'] == 'KEN', ['Country']] = 'Kenya'
df.loc[df['Continent'] == 'Middle East', ['Continent']] = 'Asia'

# Display the resulting DataFrame
#print(df[['Location', 'City', 'State', 'Country', 'Continent']])


#Ensure correct data type for each of the columns
df["City"] = df["City"].astype("string")
df["State"] = df["State"].astype("category")
df["Country"] = df["Country"].astype("category")
df["Continent"] = df["Continent"].astype("category")

In [11]:
# TOTAL HOURS

# General view
# print('Values that the Total Hours variable takes:')
# print(df['Total Hours'].value_counts(dropna=False))
# Main remarks: This variable includes many different values and NaN. Also the format needs to be aligned  (some values have commas for
# representing thousands, others have a hh:mm:ss format, some words are included, etc)

# Function for removing thousands commas and convert the last comma into a decimal separator ('.')
def process_commas(value):
    value = str(value)  # Standardize data format in the column
    if value.count(',') > 1:
        # If there's more than one comma, treat the last comma as a decimal separator
        parts = value.rsplit(',', 1)
        processed_value = parts[0].replace(',', '') + '.' + parts[1]
    else:
        # If only one comma, remove thousands comma
        processed_value = value.replace(',', '')
    return processed_value

# Apply previous function
df['Total Hours'] = df['Total Hours'].apply(process_commas)


# Function to convert hh:mm:ss to decimal hours
def convert_hours(value):
    value = str(value) # Standardize data format in the column
    try:
        # Split hours, minutes, and seconds
        hours, minutes, seconds = map(int, value.split(':'))
        # Convert minutes to fraction of hours (ignoring seconds)
        decimal_hours = hours + minutes / 60
        return round(decimal_hours, 2)  # Round to 2 decimal places
    except:
        # Return original value if it doesn’t match the expected format
        return value

# Apply previous function
df['Total Hours'] = df['Total Hours'].apply(convert_hours)


# Remove words
words_to_remove = ['SMOH', 'SFRM', 'hrs', 'hours', 'approx', 'SNEW', 'HOURS', 'Hours', 'Flight Hours (approx.)', 'since new', 'h', '*', 'Delivery time only']
regex_pattern = '|'.join(map(re.escape, words_to_remove))
df['Total Hours'] = df['Total Hours'].str.replace(regex_pattern, '', regex=True).str.strip()

# Resolve specific cases: 2.518:53, 3181:57, 10 8619, 12 Yrs / 2200 Hrs Remaining, Sub 870  TTSN 1883tt 145 since O/H
df.loc[df['Total Hours'] == '2.518:53', ['Total Hours']] = 2518.88
df.loc[df['Total Hours'] == '3181:57', ['Total Hours']] = 3181.95
df.loc[df['Total Hours'] == '10 8619', ['Total Hours']] = 10861.9
df.loc[df['Total Hours'] == '12 Yrs / 2200 Hrs Remaining', ['Total Hours']] = np.nan
df.loc[df['Total Hours'] == 'Sub 870  TTSN', ['Total Hours']] = 870
df.loc[df['Total Hours'] == '1883tt  145 since O/H', ['Total Hours']] = 145
df.loc[df['Total Hours'] == '', ['Total Hours']] = np.nan


# Check: Display values that do NOT contain only numbers (excluding valid decimals, i.e., numbers with only one dot)
non_numeric_values = df['Total Hours'][~df['Total Hours'].str.match(r'^\d+(\.\d+)?$', na=False)]
#print(non_numeric_values.values)

# Verification
#pd.set_option('display.max_rows', None)
#print(df['Total Hours'].value_counts(dropna=False))

# Convert the column to float
df['Total Hours'] = df['Total Hours'].astype('float32')


#print(df.info())

In [12]:
# TOTAL SEATS

# General view
# print('Values that the Total Seats variable takes:')
#print(df['Total Seats'].value_counts(dropna=False))
# Main remarks: This variable includes 15 values and many NaNs, convert to integer

# Convert to numeric values (integer): Int64
df['Total Seats'] = df['Total Seats'].astype('Int32')

#print(df.info()) 

In [13]:
# FLIGHT RULES

# General view
# print('Values that the FLight Rules variable takes:')
#print(df['Flight Rules'].value_counts(dropna=False))
# Main remarks: This variable includes only 3 values and many NaNs, these three values can be converted to intergers to save memory?
# 1: IFR, 2:VFR, 3:BTH

# df['Flight Rules'].replace({'IFR': '1', 'VFR': '2', 'BTH' : '3'}, inplace=True)
# df['Flight Rules']=pd.to_numeric(df['Flight Rules'], downcast='integer', errors='coerce')
# df['Flight Rules'] = df['Flight Rules'].astype('Int64')

# BTH is the same as IFR
df.loc[df['Flight Rules'] == 'BTH', ['Flight Rules']] = 'IFR'

#Convert to correct data type
df['Flight Rules'] = df['Flight Rules'].astype('category')

#print(df['Flight Rules'].value_counts(dropna=False))

In [14]:
# ENGINE 1 HOURS, ENGINE 2 HOURS, PROPELLER 1 HOURS, PROPELLER 2 HOURS

# General view
# print('Values that the Engine 1 Hours variable takes:')
# print(df['Engine 1 Hours'].value_counts(dropna=False))
# Main remarks: Many NaN values, come values include: 'hours', 'SNEW' = since new, 'SMOH' = since major overhaul, 'SPOH' = since propeller 
# overhaul, 'SCMOH' = Since Chrome Major Overhaul, separate numbers and letters and convert numbers to integers. Columna casi sin datos. 
# If it is not a missing value but instead it is not applicable use 'N/A'



# Separate maintenance type
# Function to extract numbers and letters
def separate_numbers_letters(value):
    if pd.isna(value):  # Check if the value is NaN
        return np.nan, np.nan
    numbers = ''.join(filter(str.isdigit, value))  # Extracts the numbers
    letters = ''.join(filter(str.isalpha, value))  # Extracts the letters
    return numbers, letters

# Apply the function and assign the results to new columns
df[['Eng. 1 Hours', 'Maint. Eng. 1']] = df['Engine 1 Hours'].apply(separate_numbers_letters).apply(pd.Series)
df[['Eng. 2 Hours', 'Maint. Eng. 2']] = df['Engine 2 Hours'].apply(separate_numbers_letters).apply(pd.Series)
df[['Prop. 1 Hours', 'Maint. Prop. 1']] = df['Prop 1 Hours'].apply(separate_numbers_letters).apply(pd.Series)
df[['Prop. 2 Hours', 'Maint. Prop. 2']] = df['Prop 2 Hours'].apply(separate_numbers_letters).apply(pd.Series)


# Solve values inconsistencies in Maint. columns:

# MAINT. ENG. 1
#print(df['Maint. Eng. 1'].value_counts(dropna=False))
# Convert "HOURS" and "" values into NaN in 'Maint. Eng. 1' column
df['Maint. Eng. 1'] = df['Maint. Eng. 1'].replace(['HOURS', ''], np.nan)
# Unifying TSOH = Time Since Overhaul and SOH = Since Overhaul
df.loc[df['Maint. Eng. 1'] == 'TSOH', ['Maint. Eng. 1']] = 'SOH'
# Remove T = Time in TSHSI to homogenize to the rest of the values and add T to CZI and MPI
df.loc[df['Maint. Eng. 1'] == 'TSHSI', ['Maint. Eng. 1']] = 'SHSI'
df.loc[df['Maint. Eng. 1'] == 'CZI', ['Maint. Eng. 1']] = 'SCZI'
df.loc[df['Maint. Eng. 1'] == 'MPI', ['Maint. Eng. 1']] = 'SMPI'
# TSHS with SHSI to homogenize values
df.loc[df['Maint. Eng. 1'] == 'TSHS', ['Maint. Eng. 1']] = 'SHSI'
#print(df['Maint. Eng. 1'].value_counts(dropna=False))

# MAINT. ENG. 2
#print(df['Maint. Eng. 2'].value_counts(dropna=False))
# Same changes as for Maint. Eng. 1:
df['Maint. Eng. 2'] = df['Maint. Eng. 2'].replace(['HOURS', ''], np.nan)
df.loc[df['Maint. Eng. 2'] == 'CZI', ['Maint. Eng. 2']] = 'SCZI'
df.loc[df['Maint. Eng. 2'] == 'TSHSI', ['Maint. Eng. 2']] = 'SHSI'
df.loc[df['Maint. Eng. 2'] == 'TSOH', ['Maint. Eng. 2']] = 'SOH'
df.loc[df['Maint. Eng. 2'] == 'MPI', ['Maint. Eng. 2']] = 'SMPI'
df.loc[df['Maint. Eng. 2'] == 'TSHS', ['Maint. Eng. 2']] = 'SHSI'
#print(df['Maint. Eng. 2'].value_counts(dropna=False))

# MAINT. PROP. 1
# print(df['Maint. Prop. 1'].value_counts(dropna=False))
# Same changes as for the rest of Maint.:
df['Maint. Prop. 1'] = df['Maint. Prop. 1'].replace(['HOURS', ''], np.nan)
#print(df['Maint. Prop. 1'].value_counts(dropna=False))

# MAINT. PROP. 2
#print(df['Maint. Prop. 2'].value_counts(dropna=False))
# Same changes as for the rest of Maint.:
df['Maint. Prop. 2'] = df['Maint. Prop. 2'].replace(['HOURS', ''], np.nan)
df.loc[df['Maint. Prop. 2'] == 'TSHS', ['Maint. Prop. 2']] = 'SHSI'
#print(df['Maint. Prop. 2'].value_counts(dropna=False))



# Verify the result
# print(df[['Engine 1 Hours', 'Eng. 1 Hours', 'Maint. Eng. 1']])
# print(df[['Engine 2 Hours', 'Engine 2 hours', 'Maint. Eng. 2']])
# print(df[['Prop 1 Hours', 'Prop 1 hours', 'Maint. Prop. 1']])
# print(df[['Prop 2 Hours', 'Prop 2 hours', 'Maint. Prop. 2']])


# Convert the columns to numeric values and category
df['Eng. 1 Hours'] = df['Eng. 1 Hours'].astype('float32')
df['Eng. 2 Hours'] = df['Eng. 2 Hours'].astype('float32')
df['Prop. 1 Hours'] = df['Prop. 2 Hours'].astype('float32')
df['Prop. 2 Hours'] = df['Prop. 2 Hours'].astype('float32')
df['Maint. Eng. 1'] = df['Maint. Eng. 1'].astype('category')
df['Maint. Eng. 2'] = df['Maint. Eng. 2'].astype('category')
df['Maint. Prop. 1'] = df['Maint. Prop. 1'].astype('category')
df['Maint. Prop. 2'] = df['Maint. Prop. 2'].astype('category')

#print(df.info())



In [15]:
# NATIONAL ORIGIN 

# General view
# print('Values that the National Origin variable takes:')
# print(df['National Origin'].value_counts(dropna=False))
# Main remarks: 26 values and some NULLs, some with uppercase/lowercase differences; convert to lowercase to avoid duplicates, e.g., italy
# EU replace with United States, Switzerland misspelled, Czechoslovakia = Slovakia, Britain = United Kingdom, France and Britain counted twice


# Unify value formats by using uppercase for only the first letter of each word
df['National Origin'] = df['National Origin'].str.title()

# Ammend values
df['National Origin'].replace({'Eu': 'EU', 'Swtizerland': 'Switzerland', 'Czechoslovakia' : 'Czech Republic', 'Britain':'United Kingdom'}, inplace=True)


# The same degree of precision is not achieved by EU value, as there are 59 EU values, the model are going to be studied to ammend the specific country in which they 
# are produced

# Filter rows with EU into National Origin column
df_filtered = df[df['National Origin'] == 'EU']
# Show 'Model' and 'Make' for those rows
#print(df_filtered[['Model', 'Make', 'National Origin']])

# Ammend values
df.loc[df['Model'] == 'As 350Ba', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Bk 117 B-2', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'Ec 135P2', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'As 350B-2', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 120B', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 365N-1', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 355F-2', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 355N', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 135', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'Ec 120', ['National Origin']] = 'France'
df.loc[df['Model'] == 'H125', ['National Origin']] = 'France'
df.loc[df['Model'] == 'H130', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 350B-3', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 130B4', ['National Origin']] = 'France'
df.loc[df['Model'] == 'H145', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'Ec 145', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'Ec 135P2+', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'As 355Np', ['National Origin']] = 'Germany'
df.loc[df['Model'] == 'As 350B', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 225', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 365N-2', ['National Origin']] = 'France'
df.loc[df['Model'] == 'A320', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 155B1', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Ec 155B', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 355F-2', ['National Origin']] = 'France'
df.loc[df['Model'] == 'As 350Sd2', ['National Origin']] = 'France'
df.loc[df['Model'] == 'Bo 105Cbs-4', ['National Origin']] = 'Germany'

# Convert the columns to category
df['National Origin'] = df['National Origin'].astype('category')

#print(df['National Origin'].value_counts(dropna=False))

C:\Users\Ángela Martín Milán\AppData\Local\Temp\ipykernel_40288\2268868285.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['National Origin'].replace({'Eu': 'EU', 'Swtizerland': 'Switzerland', 'Czechoslovakia' : 'Czech Republic', 'Britain':'United Kingdom'}, inplace=True)


In [16]:
# Remove no longer necessary columns 
# print(df.columns)
df.drop(columns = ['Category', 'Engine 1 Hours', 'Engine 2 Hours', 'Prop 1 Hours', 'Prop 2 Hours', 'Location'], axis=1, inplace=True)
# print(df.columns)

# Rename reamining ones (only required for Category)
df.rename(columns={'category': 'Category'}, inplace=True)
# print(df.columns)

# Reorder columns
new_order = ['Price', 'Currency', 'Year', 'Condition', 'Total Hours', 'Total Seats','Flight Rules', 'Category', 'Propulsion',
             'No. Engines', 'Make', 'Model', 'City', 
             'State', 'Country', 'Continent', 'National Origin', 'Eng. 1 Hours', 'Maint. Eng. 1',
             'Eng. 2 Hours', 'Maint. Eng. 2', 'Prop. 1 Hours', 'Maint. Prop. 1',
             'Prop. 2 Hours', 'Maint. Prop. 2', 'text']
df = df[new_order]

#print(df.info())


# Save the clean DataFrame linto a CSV file
df.to_csv(r'C:\Users\Ángela Martín Milán\Desktop\TFM\clean.csv', index=False)